# 自然语言推断：微调BERT

In [1]:
import json
import multiprocessing
import os
import torch
from torch import nn
from d2l import torch as d2l
from multiprocessing.dummy import Pool as ThreadPool

## [**加载预训练的BERT**]

In [2]:
d2l.DATA_HUB['bert.base'] = (d2l.DATA_URL + 'bert.base.torch.zip',
                             '225d66f04cae318b841a13d32af3acc165f253ac')
d2l.DATA_HUB['bert.small'] = (d2l.DATA_URL + 'bert.small.torch.zip',
                              'c72329e68a732bef0452e4b96a1c341c8910f81f')

| 参数                     | 含义                     | 典型值            |
| ---------------------- | ---------------------- | -------------- |
| **`pretrained_model`** | 预训练模型名称（如'bert.small'） | `'bert.small'` |
| **`num_hiddens`**      | BERT隐藏层维度              | 256            |
| **`ffn_num_hiddens`**  | 前馈网络隐藏层维度              | 512            |
| **`num_heads`**        | 多头注意力头数                | 4              |
| **`num_layers`**       | Transformer层数          | 2              |
| **`dropout`**          | Dropout比率              | 0.2            |
| **`max_len`**          | 最大序列长度                 | 512            |
| **`devices`**          | 计算设备列表                 | `[cuda:0]`     |


In [3]:
def load_pretrained_model(pretrained_model, num_hiddens, ffn_num_hiddens,
                          num_heads, num_layers, dropout, max_len, devices):
    # 1. 下载并解压预训练模型：从d2l服务器下载预训练的BERT模型文件（包含词汇表和参数），解压到本地缓存目录，返回路径
    data_dir = d2l.download_extract(pretrained_model)
    '''
    2. 创建空词汇表并加载预定义词汇
    先创建空Vocab对象：因为要用预训练模型的词汇表，而非现场构建
    加载词汇表：从vocab.json读取词元到索引的映射列表
        文件内容示例：["<pad>","<unk>","the","a",...,"bert"]
    '''
    vocab = d2l.Vocab()
    vocab.idx_to_token = json.load(open(os.path.join(data_dir,
        'vocab.json')))
    # 3. 构建反向映射
    # token_to_idx：从词元到索引的字典，用于快速查找
    # 必要性：模型输入需要索引，而vocab.json只保存了正向映射
    vocab.token_to_idx = {token: idx for idx, token in enumerate(
        vocab.idx_to_token)}
    '''
    4. 初始化BERT模型结构
    关键：len(vocab)确保模型词汇表大小与预训练一致
    超参数匹配 ：所有维度参数必须与预训练时完全相同，否则无法加载权重
        num_hiddens=256：隐藏层维度
        num_heads=4：4头注意力
        num_layers=2：2层Transformer
        key_size=256, query_size=256, value_size=256：Q/K/V投影维度
        ffn_num_input=256, ffn_num_hiddens=512：前馈网络输入/隐藏维度
    '''
    bert = d2l.BERTModel(len(vocab), num_hiddens, norm_shape=[256],
                         ffn_num_input=256, ffn_num_hiddens=ffn_num_hiddens,
                         num_heads=4, num_layers=2, dropout=0.2,
                         max_len=max_len, key_size=256, query_size=256,
                         value_size=256, hid_in_features=256,
                         mlm_in_features=256, nsp_in_features=256)
    '''
    5. 加载预训练参数
    pretrained.params：保存预训练权重的PyTorch序列化文件
    torch.load()：从磁盘读取权重字典
    load_state_dict()：将权重加载到模型中
    要求：模型结构必须与预训练时完全一致，否则会报错
    严格匹配：层名、参数形状必须一一对应
    '''
    bert.load_state_dict(torch.load(os.path.join(data_dir,
                                                 'pretrained.params')))
    # 6. 返回模型和词汇表
    return bert, vocab

In [4]:
devices = d2l.try_all_gpus()
bert, vocab = load_pretrained_model(
    'bert.small', num_hiddens=256, ffn_num_hiddens=512, num_heads=4,
    num_layers=2, dropout=0.1, max_len=512, devices=devices)

## [**微调BERT的数据集**]


```Python
原始 dataset:
(
    ["A man is walking", "The woman sings", ...],      # premises
    ["A person moves", "The lady is singing", ...],    # hypotheses
    [0, 1, 2, ...]                                    # labels
)

↓ dataset[:2]

[
    ["A man is walking", "The woman sings", ...],      # 第一层遍历
    ["A person moves", "The lady is singing", ...]
]

↓ [s.lower() for s in sentences]

[
    ["a man is walking", "the woman sings", ...],
    ["a person moves", "the lady is singing", ...]
]

↓ d2l.tokenize(...)

[
    [["a", "man", "is", "walking"], ["the", "woman", "sings"], ...],
    [["a", "person", "moves"], ["the", "lady", "is", "singing"], ...]
]

↓ *（解压）

list1 = [["a", "man", ...], ["the", "woman", ...], ...]
list2 = [["a", "person", ...], ["the", "lady", ...], ...]

↓ zip(list1, list2)

[
    (["a", "man", "is", "walking"], ["a", "person", "moves"]),
    (["the", "woman", "sings"], ["the", "lady", "is", "singing"]),
    ...
]

↓ 最终列表推导式

[
    [["a", "man", "is", "walking"], ["a", "person", "moves"]],
    [["the", "woman", "sings"], ["the", "lady", "is", "singing"]],
    ...
]
```
```python
# 对于前提"A man walks" + 假设"A person moves"
token_ids: [101, 1037, 2153, 5231, 102, 1037, 2153, 5231, 102, 0, 0, ...]
segments:  [0,   0,    0,    0,    0,   1,    1,    1,    1,   0, 0, ...]
valid_len:  9  (实际词元数)
label:      0  (蕴含)
```
**为什么需要这种复杂结构？**

BERT要求输入包含：
1. 词元ID：词语的数字表示
2. 段落段ID：区分句子A和句子B
3. 有效长度：避免注意力计算到填充部分
```python
token_ids: [101, 1037, 2153, 5231, 102, 1037, 2153, 5231, 102, 0, 0, ...]
            # [CLS]  a   man walks [SEP]  a  person moves [SEP] [PAD]...
segments:  [0,   0,    0,    0,     0,   1,   1,    1,    1,   0, 0, ...]
            # 前提部分为0          假设部分为1      填充部分为0
valid_len: 9  # 实际词元数（不含填充）
```
```python
输入: (['a', 'man', 'walks'], ['a', 'person', 'moves'])

1. 截断（本例无需）
   → p_tokens: ['a', 'man', 'walks']
   → h_tokens: ['a', 'person', 'moves']

2. get_tokens_and_segments
   → tokens:  ['<cls>', 'a', 'man', 'walks', '<sep>', 'a', 'person', 'moves', '<sep>']
   → segments: [0, 0, 0, 0, 0, 1, 1, 1, 1]

3. 转ID并填充至max_len=12
   → token_ids: [101, 1037, 2153, 5231, 102, 1037, 2153, 5231, 102, 0, 0, 0]
   → segments:  [0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]

4. valid_len = 9

返回: ([101, 1037, 2153, 5231, 102, 1037, 2153, 5231, 102, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
       9)
```

In [5]:
class SNLIBERTDataset(torch.utils.data.Dataset):
    '''
    dataset：原始SNLI数据元组 (premises,hypotheses,labels)
    max_len：BERT输入的最大序列长度（如128或512）
    vocab：BERT词汇表（若不提供则现场构建，但通常使用预训练词表）
    '''
    def __init__(self, dataset, max_len, vocab=None):
        # 1. 词元化所有句子对：结果：每个元素是[前提词元列表,假设词元列表]
        '''
        层1：提取前提和假设列表dataset[:2]
            dataset是元组 (premises,hypotheses,labels)
            dataset[:2]切片后得到[premises,hypotheses]（前两个元素）
        层2：遍历两个句子列表for sentences in dataset[:2]
            第一次迭代：sentences=premises（前提句子列表）
            第二次迭代：sentences=hypotheses（假设句子列表）
        层3：小写化处理[s.lower() for s in sentences]
            作用：将每个句子的所有字符转为小写
            原因：统一大小写，减少词汇表冗余，提升模型泛化能力
            示例："A man is walking"→"a man is walking"
        层4：批量分词
            d2l.tokenize([s.lower() for s in sentences])
            对两个列表分别调用：tokenize将句子字符串拆分为词元列表
            第一次调用：对premises分词
            输入：["a man is walking","the woman sings"]
            输出：[["a","man","is","walking"],["the","woman","sings"]]
            第二次调用：对hypotheses分词
            输入：["a person moves","the lady is singing"]
            输出：[["a","person","moves"], ["the","lady","is","singing"]]
        层5：解压操作（最关键）*[d2l.tokenize(...)]
            *是"解压运算符"，将列表中的元素作为独立参数传递
            结果：相当于将两个分词结果列表平铺为两个独立参数
            # 等价于：
            zip(
                [["a", "man", ...], ["the", "woman", ...]],  # 前提词元列表
                [["a", "person", ...], ["the", "lady", ...]]  # 假设词元列表
            )
        层6：按样本配对for p_tokens, h_tokens in zip(...)
            zip将两个列表的对应元素配对
            第一次迭代：p_tokens=["a","man","is","walking"],h_tokens=["a","person","moves"]
            第二次迭代：p_tokens=["the","woman","sings"],h_tokens=["the","lady","is","singing"]
        层7：构建配对列表[[p_tokens, h_tokens] for ...]
            最终结构：每个元素是 [前提词元,假设词元]的列表
        '''
        all_premise_hypothesis_tokens = [[
            p_tokens, h_tokens] for p_tokens, h_tokens in zip(
            *[d2l.tokenize([s.lower() for s in sentences])
              for sentences in dataset[:2]])]
        '''
        2. 标签张量化
        dataset[2]：整数标签列表（0=蕴含,1=矛盾,2=中性）
        转换为PyTorch张量，便于后续计算损失
        '''
        self.labels = torch.tensor(dataset[2])
        # 3. 保存配置：保存词汇表和最大长度供后续使用
        self.vocab = vocab
        self.max_len = max_len
        '''
        4. 预处理为BERT输入格式（元组解包）
        all_token_ids：合并前提和假设并填充后的词元ID序列
            格式：[CLS]+premise+[SEP]+hypothesis+[SEP]+[PAD]...
            形状：(num_examples,max_len)
        all_segments：段落ID（区分前提和假设）
            前提部分标记为0，假设部分标记为1
            形状：(num_examples,max_len)
        valid_lens：有效长度（不含填充的实际词元数）
            用于注意力掩码
            形状：(num_examples,)
        '''
        (self.all_token_ids, self.all_segments,
         self.valid_lens) = self._preprocess(all_premise_hypothesis_tokens)
        # 5. 打印统计
        print('read ' + str(len(self.all_token_ids)) + ' examples')

    def _preprocess(self, all_premise_hypothesis_tokens):
        '''
        1. 创建进程池
        作用：创建一个包含4个worker进程的进程池
        目的：加速大规模数据预处理（SNLI有55万样本），利用多核CPU并行计算
        原理：将数据分片后分配给多个进程同时处理，避免单进程CPU瓶颈
        '''
        pool = ThreadPool(4)
        # pool = multiprocessing.Pool(4)  # 使用4个进程
        '''
        2. 并行映射处理
        self._mp_worker：多进程工作函数（未显示但可推断），处理单个样本：
        输入：[premise_tokens,hypothesis_tokens]（一个词元对）
        处理：
            添加特殊符号：[CLS]、[SEP]
            转换为词元ID：self.vocab[...]
            生成段落ID：前提部分为0，假设部分为1
            截断/填充到max_len
            计算有效长度（不含填充）
        返回：(token_ids,segments,valid_len)三元组
        pool.map()：将all_premise_hypothesis_tokens列表分片给各进程并行处理
        效果：4个进程同时处理不同子集，速度提升近4倍
        '''
        out = pool.map(self._mp_worker, all_premise_hypothesis_tokens)
        '''
        3. 提取并行结果
        out是每个进程的返回结果列表，每个元素是(token_ids,segments,valid_len)
        列表推导式：将三个组件分别提取到独立列表
        token_ids：词元ID列表
            列表推导式：遍历out中每个元组
            模式匹配：token_ids,segments,valid_len解包每个元组
            结果：只保留第一个元素（token_ids），构成新列表
            形状：[(样本0的token_ids),(样本1的token_ids),...]
            后续转换：torch.tensor(all_token_ids)→形状(num_examples,max_len)
        segments：段落ID列表
            逻辑同上：遍历并提取第二个元素（segments）
            结果：所有样本的段落ID列表
            后续转换：torch.tensor(all_segments)→形状(num_examples,max_len)
        valid_len：有效长度（整数）
            遍历并提取第三个元素（valid_len）
            结果：所有样本的有效长度列表
            后续转换：torch.tensor(valid_lens)→形状(num_examples,)
        '''
        all_token_ids = [token_ids for token_ids, segments, valid_len in out]
        all_segments  = [segments  for token_ids, segments, valid_len in out]
        valid_lens    = [valid_len for token_ids, segments, valid_len in out]
        '''
        4. 转换为张量
        dtype=torch.long：词元ID和段落ID需为整数类型
        返回值：
            all_token_ids：(num_examples,max_len)-所有样本的词元ID矩阵
            all_segments：(num_examples,max_len)-段落ID矩阵
            valid_lens：(num_examples,)-每个样本的有效长度向量
        '''
        return (torch.tensor(all_token_ids, dtype=torch.long),
                torch.tensor(all_segments, dtype=torch.long),
                torch.tensor(valid_lens))

    def _mp_worker(self, premise_hypothesis_tokens):
        # 1. 解包词元对：输入：premise_hypothesis_tokens是[premise_tokens,hypothesis_tokens]列表
        # 示例：[['a','man','walks'],['a','person','moves']]
        p_tokens, h_tokens = premise_hypothesis_tokens
        '''
        2. 截断词元对
        作用：确保前提和假设的总长度（含特殊符号）不超过max_len-3（保留空间给[CLS]和两个[SEP]）
        策略：从较长一方尾部截断，保持两者相对完整
        '''
        self._truncate_pair_of_tokens(p_tokens, h_tokens)
        '''
        3. 获取带特殊符号的词元和段落ID
        tokens：
            格式：[CLS]+p_tokens+[SEP]+h_tokens+[SEP]
            示例：['<cls>','a','man','walks','<sep>','a','person','moves','<sep>']
        segments *：分段标记
        前提部分（含CLS和第一个SEP）标记为0
        假设部分（含第二个SEP）标记为1
        示例：[0,0,0,0,0,1,1,1,1]
        '''
        tokens, segments = d2l.get_tokens_and_segments(p_tokens, h_tokens)
        '''
        4. 转换为词元ID并填充
        self.vocab[tokens]：将词元列表转为索引列表。示例：[101,1037,2153,5231,102,1037,2153,5231,102]
        +[self.vocab['<pad>']]*(self.max_len-len(tokens))：
            在末尾填充<pad>至最大长度：确保所有样本长度一致，便于批量处理
        '''
        token_ids = self.vocab[tokens] + [self.vocab['<pad>']] \
                             * (self.max_len - len(tokens))
        # 5. 填充段落ID：填充部分也标记为0（或其他值，通常不影响，因为会被valid_len忽略）
        segments = segments + [0] * (self.max_len - len(segments))
        # 6. 记录有效长度： 实际词元数（不含填充），用于后续注意力掩码
        valid_len = len(tokens)
        '''
        7. 返回
        token_ids：填充后的词元ID列表，长度=max_len
        segments：填充后的段落ID列表，长度=max_len
        valid_len：整数，表示实际词元数量
        '''
        return token_ids, segments, valid_len

    def _truncate_pair_of_tokens(self, p_tokens, h_tokens):
        # 为BERT输入中的'<CLS>'、'<SEP>'和'<SEP>'词元保留位置
        '''
        max_len-3：为三个特殊符号预留位置
        [CLS]：序列开头标记（1个）
        第一个[SEP]：分隔前提和假设（1个）
        第二个[SEP]：序列结尾标记（1个）
        示例：若max_len=128，则前提+假设的实际词元数不能超过125
        '''
        while len(p_tokens) + len(h_tokens) > self.max_len - 3:
            '''
            平衡截断策略
            策略：总是从较长的句子中删除词元
            目的：尽可能保留两句话的 相对完整信息
            pop()：从列表末尾删除元素（删除最不重要的尾部词元）
            '''
            if len(p_tokens) > len(h_tokens):
                p_tokens.pop() # 删除较长句子的最后一个词元
            else:
                h_tokens.pop() # 删除较长句子的最后一个词元
    '''
    特征元组：
        self.all_token_ids[idx]：样本的词元ID序列（已填充至max_len）
        self.all_segments[idx]：段落ID序列（区分前提/假设）
        self.valid_lens[idx]：有效长度（不含填充）
    标签：self.labels[idx]：类别标签（0/1/2）
    '''
    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx]), self.labels[idx]
    '''
    作用：
        当调用len(dataset)时自动触发
        返回数据集中样本的总数
        使用self.all_token_ids的长度作为样本数（因为每个样本对应一个token ID序列）
    在PyTorch中的重要性：
        DataLoader 依赖此方法确定数据批次的数量和边界
        与__getitem__配合，构成完整的数据集接口
    '''
    def __len__(self):
        return len(self.all_token_ids)

In [6]:
# 如果出现显存不足错误，请减少“batch_size”。在原始的BERT模型中，max_len=512
'''
原始文本 → read_snli → 分词 → SNLIBERTDataset → 多进程转ID → 填充/截断 → 张量化 → DataLoader → 批量数据 → BERT模型
1. 设置超参数
batch_size=512：每个训练批次的样本数量（较大批次提升GPU利用率）
max_len=128：BERT输入序列的最大长度（含特殊符号），截断超长句子
num_workers：数据加载进程数，自动获取CPU核心数以加速数据读取
'''
batch_size, max_len, num_workers = 32, 128, 0
# 2. 下载SNLI数据集
data_dir = '..\\data\\snli_1.0'
'''
3. 创建训练集对象
d2l.read_snli(data_dir,True)：读取训练集原始数据
返回(premises,hypotheses,labels)元组
SNLIBERTDataset(...)：初始化BERT格式数据集
多层预处理：
    分词+小写化
    多进程并行转ID（4个worker）
    添加特殊符号（[CLS]/[SEP]）
    生成段落ID
    填充/截断至max_len
    计算有效长度
'''
train_set = SNLIBERTDataset(d2l.read_snli(data_dir, True), max_len, vocab)
'''
4. 创建测试集对象
    read_snli(...,False)：读取测试集
    复用vocab：确保训练集和测试集使用相同的词表
    无shuffle：测试集不需要打乱
'''
test_set = SNLIBERTDataset(d2l.read_snli(data_dir, False), max_len, vocab)
'''
5. 创建训练数据迭代器
    DataLoader：PyTorch标准数据加载器
    shuffle=True：每轮训练打乱数据顺序，防止模型记忆数据模式
    多worker加载：在训练时并行读取数据，避免GPU等待
'''
train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True,
                                   num_workers=0)
'''
6. 创建测试数据迭代器
    shuffle=False：默认不打乱，保持原始顺序便于评估
    相同batch_size：测试时批量大小可不同，但通常保持一致
'''
test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                  num_workers=0)

read 549367 examples
read 9824 examples


## 微调BERT


```python
# 假设 batch_size=32, seq_len=128
input: (tokens_X, segments_X, valid_lens_x)
       tokens_X: (32, 128)      # [CLS] premise [SEP] hypothesis [SEP] [PAD]...
       segments_X: (32, 128)    # 0 0 0 0 0 1 1 1 1 0 0 0...
       valid_lens_x: (32,)      # [45, 52, 38, ...]

↓ self.encoder (3层Transformer)
encoded_X: (32, 128, 256)      # 每个位置的上下文表示

↓ 提取 [CLS] (位置0)
cls_rep: (32, 256)              # [CLS]聚合了整个句子的语义

↓ self.hidden (MLP)
hidden_rep: (32, 256)           # 任务特定的特征变换

↓ self.output (Linear)
logits: (32, 3)                 # 三分类输出
```

In [7]:
class BERTClassifier(nn.Module):
    '''
    模块复用策略
        self.encoder：复用预训练BERT的2层Transformer编码器，提取上下文语义表示
        self.hidden：复用BERT的隐藏层前馈网络（通常用于MLM任务），作为特征转换层
        self.output：新增任务特定输出层
        输入维度：256（BERT隐藏层维度）
        输出维度：3（SNLI三分类：蕴含/矛盾/中性）
    '''
    def __init__(self, bert):
        super(BERTClassifier, self).__init__() 
        self.encoder = bert.encoder # BERT的Transformer编码器
        self.hidden = bert.hidden # BERT的隐藏层（MLM任务头）
        self.output = nn.Linear(256, 3) # 自定义输出层（3分类）

    def forward(self, inputs):
        '''
        输入解包
            inputs：元组(token_ids,segments,valid_lens)
            tokens_X：词元ID张量，形状(batch_size,seq_len)
            segments_X：段落ID张量，区分前提/假设
            valid_lens_x：有效长度张量，用于注意力掩码
        '''
        tokens_X, segments_X, valid_lens_x = inputs
        '''
        编码器处理
            作用：通过12层Transformer，生成上下文感知的词元表示
            输出形状：(batch_size,seq_len,hidden_dim)，如(32,128,256)
        '''
        encoded_X = self.encoder(tokens_X, segments_X, valid_lens_x)
        '''
        特征提取与分类
        提取[CLS]表示
            encoded_X[:,0,:]：选取所有样本中第0个位置（[CLS]词元）的向量
            : ：批量维度，选取所有样本
            0 ：序列维度，固定选取[CLS]位置
            : ：特征维度，选取全部256维
            为什么选[CLS]？ ：BERT预训练时，[CLS]聚合了整个序列的语义，是分类任务的标准做法
        通过隐藏层
            self.hidden(...)：将[CLS]表示通过MLP进行非线性变换
            作用：从通用语义空间映射到任务特定特征空间
        最终分类
            self.output(...)：输出3个类别的logits
            结果形状：(batch_size,3)
        '''
        return self.output(self.hidden(encoded_X[:, 0, :]))

In [8]:
net = BERTClassifier(bert)

In [ ]:
# ===== 1. 超参数调整（激进快速模式） =====
batch_size, max_len = 16, 64  # 批量减半，长度减半
num_workers = 2  # 必须开启多进程
num_epochs = 1   # 只跑1轮！
lr = 1e-3        # 提高学习率加速收敛（仅用于测试）

# ===== 2. 数据子集（只用1000个样本）=====
def get_subset_dataset(dataset, max_samples=1000):
    """提取前max_samples个样本用于快速验证"""
    premises, hypotheses, labels = dataset
    return (premises[:max_samples], 
            hypotheses[:max_samples], 
            labels[:max_samples])

# 创建迷你数据集
mini_train_data = get_subset_dataset(d2l.read_snli(data_dir, True), 1000)
mini_test_data = get_subset_dataset(d2l.read_snli(data_dir, False), 200)

train_set = SNLIBERTDataset(mini_train_data, max_len, vocab)
test_set = SNLIBERTDataset(mini_test_data, max_len, vocab)

# ===== 3. 创建DataLoader（保持多进程）=====
train_iter = torch.utils.data.DataLoader(
    train_set, batch_size, shuffle=True, 
    num_workers=num_workers, pin_memory=True
)
test_iter = torch.utils.data.DataLoader(
    test_set, batch_size, shuffle=False, 
    num_workers=num_workers, pin_memory=True
)

# ===== 4. 模型初始化（使用tiny版BERT）=====
try:
    # 如果有GPU，强制使用GPU
    devices = [torch.device('cuda:0')]
    print("✓ 使用GPU加速")
except:
    devices = [torch.device('cpu')]
    print("⚠ 使用CPU，将非常慢")

# 使用BERT-tiny（如果可用）或冻结部分层
# net = d2l.BERTClassifier(...)  # 您的模型定义

# ===== 5. 训练（启用混合精度加速）=====
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')

# PyTorch 2.0+ 可用torch.compile加速
if hasattr(torch, 'compile'):
    net = torch.compile(net)
    print("✓ 启用torch.compile加速")

# 开始训练
print("="*50)
print(f"快速验证开始！数据量: {len(train_set)}, 设备: {devices[0]}")
print("="*50)

d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

read 1000 examples
read 200 examples
✓ 使用GPU加速
✓ 启用torch.compile加速
快速验证开始！数据量: 1000, 设备: cuda:0


In [9]:
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os

# ==================== 第0步：Windows环境强制设置 ====================
# 必须在所有导入之后、任何代码之前执行
if __name__ == '__main__':  # ← 这行必须有！
    multiprocessing.freeze_support()
    
    # ==================== 第1步：超参数（极限压缩） ====================
    batch_size, max_len = 8, 32  # 进一步压缩：8×32 = 比原来快10倍
    num_epochs = 1
    lr = 1e-3
    
    # ==================== 第2步：数据子集（仅50条！） ====================
    def get_tiny_dataset(dataset, n=50):
        """极致压缩：只用50条样本，30秒内完成"""
        premises, hypotheses, labels = dataset
        return (premises[:n], hypotheses[:n], labels[:n])
    
    print("🔄 开始加载数据...")
    start_time = time.time()
    
    # 读取完整数据集结构（但只取前50条）
    full_train = d2l.read_snli(data_dir, True)
    full_test = d2l.read_snli(data_dir, False)
    
    # 创建超迷你数据集
    tiny_train = get_tiny_dataset(full_train, 50)
    tiny_test = get_tiny_dataset(full_test, 20)
    
    print(f"✓ 数据加载完成，耗时 {time.time() - start_time:.2f} 秒")
    print(f"训练集: {len(tiny_train[0])}条, 测试集: {len(tiny_test[0])}条")
    
    # ==================== 第3步：单进程安全模式 ====================
    print("\n🔄 开始预处理（单进程模式，带进度显示）...")
    
    # 修改SNLIBERTDataset的_preprocess（临时补丁）
    original_preprocess = SNLIBERTDataset._preprocess
    
    def safe_preprocess(self, tokens_list):
        print(f"   正在处理 {len(tokens_list)} 个样本...")
        out = []
        for i, tokens in enumerate(tokens_list):
            out.append(self._mp_worker(tokens))
            # 每10个样本打印一次进度
            if (i + 1) % 10 == 0:
                print(f"   已处理 {i+1}/{len(tokens_list)}")
        print("   ✓ 预处理完成")
        
        all_token_ids = [t for t, s, v in out]
        all_segments = [s for t, s, v in out]
        valid_lens = [v for t, s, v in out]
        
        return (torch.tensor(all_token_ids, dtype=torch.long),
                torch.tensor(all_segments, dtype=torch.long),
                torch.tensor(valid_lens))
    
    # 临时替换方法
    SNLIBERTDataset._preprocess = safe_preprocess
    
    start_time = time.time()
    train_set = SNLIBERTDataset(tiny_train, max_len, vocab)
    test_set = SNLIBERTDataset(tiny_test, max_len, vocab)
    print(f"✓ 预处理完成，总耗时 {time.time() - start_time:.2f} 秒")
    
    # ==================== 第4步：DataLoader（num_workers=0！） ====================
    # 关键：Windows下必须设为0，否则卡死
    train_iter = DataLoader(train_set, batch_size=batch_size, 
                           shuffle=True, num_workers=0, pin_memory=True)
    test_iter = DataLoader(test_set, batch_size=batch_size, 
                          shuffle=False, num_workers=0, pin_memory=True)
    
    print(f"\n✓ DataLoader创建成功")
    print(f"训练批次数量: {len(train_iter)}, 测试批次数量: {len(test_iter)}")
    
    # ==================== 第5步：模型与设备检查 ====================
    print("\n🔄 初始化模型...")
    
    # 检查设备
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print(f"✓ 检测到GPU: {torch.cuda.get_device_name(0)}")
        print(f"  显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    else:
        device = torch.device('cpu')
        print("⚠ 未检测到GPU，使用CPU训练（会非常慢）")
    
    devices = [device]
    
    # 初始化模型（假设net已定义）
    # net = d2l.BERTClassifier(...)
    net = net.to(device)
    
    # 统计参数量
    total_params = sum(p.numel() for p in net.parameters())
    trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print(f"  总参数量: {total_params:,}, 可训练: {trainable_params:,}")
    
    # ==================== 第6步：训练前验证（关键！） ====================
    print("\n🔍 执行前向传播测试...")
    
    # 取一个batch测试
    batch = next(iter(train_iter))
    (token_ids, segments, valid_lens), labels = batch
    
    # 移动到设备
    token_ids = token_ids.to(device)
    segments = segments.to(device)
    valid_lens = valid_lens.to(device)
    labels = labels.to(device)
    
    print(f"  输入形状: token_ids={token_ids.shape}, segments={segments.shape}")
    print(f"  标签形状: {labels.shape}")
    
    # 前向传播
    net.eval()
    with torch.no_grad():
        output = net(token_ids, segments, valid_lens)
    
    print(f"  输出形状: {output.shape}")
    print(f"  预期形状: {labels.shape}")
    
    # 检查是否匹配
    assert output.shape[0] == labels.shape[0], "批次大小不匹配！"
    assert output.shape[1] == 3, "输出类别数不是3！"
    print("✓ 前向传播测试通过")
    
    # ==================== 第7步：训练一个batch测试 ====================
    print("\n🔍 执行单批次训练测试...")
    
    net.train()
    trainer = torch.optim.Adam(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    
    # 训练一个batch
    trainer.zero_grad()
    output = net(token_ids, segments, valid_lens)
    loss_val = loss_fn(output, labels)
    
    print(f"  损失值: {loss_val.item():.4f}")
    
    loss_val.backward()
    trainer.step()
    
    # 检查梯度
    grad_norm = torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
    print(f"  梯度范数: {grad_norm:.4f}")
    print("✓ 单批次训练测试通过")
    
    # ==================== 第8步：完整训练 ====================
    print("\n" + "="*50)
    print("🚀 开始完整快速训练（1 epoch）")
    print("="*50)
    
    start_time = time.time()
    
    # 使用d2l的train_ch13
    d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)
    
    print(f"\n✅ 快速验证完成！总耗时: {time.time() - start_time:.2f} 秒")
    
    # ==================== 第9步：验证结果 ====================
    print("\n📊 结果验证:")
    # 计算准确率
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_iter:
            (token_ids, segments, valid_lens), labels = batch
            token_ids = token_ids.to(device)
            segments = segments.to(device)
            valid_lens = valid_lens.to(device)
            labels = labels.to(device)
            
            output = net(token_ids, segments, valid_lens)
            pred = output.argmax(dim=1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    print(f"  测试集准确率: {accuracy:.2%} (随机基线: 33.33%)")
    
    if accuracy > 0.4:
        print("  🎉 模型正在学习！快速验证通过")
    else:
        print("  ⚠  准确率偏低，可能需要更多训练")

🔄 开始加载数据...
✓ 数据加载完成，耗时 15.22 秒
训练集: 50条, 测试集: 20条

🔄 开始预处理（单进程模式，带进度显示）...
   正在处理 50 个样本...
   已处理 10/50
   已处理 20/50
   已处理 30/50
   已处理 40/50
   已处理 50/50
   ✓ 预处理完成
read 50 examples
   正在处理 20 个样本...
   已处理 10/20
   已处理 20/20
   ✓ 预处理完成
read 20 examples
✓ 预处理完成，总耗时 0.00 秒

✓ DataLoader创建成功
训练批次数量: 7, 测试批次数量: 3

🔄 初始化模型...
✓ 检测到GPU: NVIDIA GeForce RTX 4060 Laptop GPU
  显存: 8.0GB
  总参数量: 16,613,635, 可训练: 16,613,635

🔍 执行前向传播测试...
  输入形状: token_ids=torch.Size([8, 32]), segments=torch.Size([8, 32])
  标签形状: torch.Size([8])


TypeError: forward() takes 2 positional arguments but 4 were given

In [ ]:
lr, num_epochs = 1e-4, 5
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,
    devices)